IRQA 시스템

1. 사용자가 입력한 질문과 유사한 질문 Top-N개를 추출
2. 문장 유사도 판별 언어모델을 활용해 진짜 유사한 질문을 탐색

2가지 단계를 거쳐 사용자 질문에 대한 답을 해주는 챗봇 시스템

# IRQA 챗봇 제작하기

In [ ]:
!pip install transformers

In [ ]:
import pandas as pd

!git clone https://github.com/songys/Chatbot_data.git

data = pd.read_csv('/content/Chatbot_data/ChatbotData.csv')
data.head()

Cloning into 'Chatbot_data'...
remote: Enumerating objects: 69, done.
remote: Counting objects: 100% (21/21), done.
remote: Compressing objects: 100% (17/17), done.
remote: Total 69 (delta 18), reused 4 (delta 4), pack-reused 48 (from 1)
Receiving objects: 100% (69/69), 398.65 KiB | 7.12 MiB/s, done.
Resolving deltas: 100% (35/35), done.


,Q,A,label
0,12시 땡!,하루가 또 가네요.,0
1,1지망 학교 떨어졌어,위로해 드립니다.,0
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.,0
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.,0
4,PPL 심하네,눈살이 찌푸려지죠.,0


In [ ]:
import torch
from transformers import AutoTokenizer, AutoModel

device= torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

MODEL_NAME = "bert-base-multilingual-cased"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModel.from_pretrained(MODEL_NAME)
model.to(device)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(119547, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSdpaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=Fals

In [ ]:
chatbot_Q = data['Q'].values
chatbot_A = data['A'].values

print(chatbot_Q[:])
print(chatbot_A[:])

['12시 땡!' '1지망 학교 떨어졌어' '3박4일 놀러가고 싶다' ... '흑기사 해주는 짝남.'
 '힘든 연애 좋은 연애라는게 무슨 차이일까?' '힘들어서 결혼할까봐']
['하루가 또 가네요.' '위로해 드립니다.' '여행은 언제나 좋죠.' ... '설렜겠어요.'
 '잘 헤어질 수 있는 사이 여부인 거 같아요.' '도피성 결혼은 하지 않길 바라요.']


# CLS 토큰을 얻기 위한 함수

In [ ]:
def get_cls_token(sent_a):
  model.eval()
  tokenized_sent = tokenizer(
      sent_a,
      return_tensors='pt',
      truncation=True,
      add_special_tokens=True,
      max_length=128
  ).to(device)
  with torch.no_grad(): # 그래디언트 계산 비활성화
    outputs = model(
        input_ids = tokenized_sent['input_ids'],
        attention_mask = tokenized_sent['attention_mask'],
        token_type_ids = tokenized_sent['token_type_ids']
    )
    logits = outputs.last_hidden_state[:,0,:].detach().cpu().numpy()
    return logits

In [ ]:
# 전체 질문에 대한 vector 정보 저장

chatbot_q_vectors = {}
for i, q in enumerate(chatbot_Q):
  chatbot_q_vectors[i] = get_cls_token(q)

In [ ]:
# 전체 질문과 유사도 검사를 통한 Top-n개를 변환하는 코드

import numpy as np

In [ ]:
def custom_cosine_similarity(a,b):
    numerator = np.dot(a,b.T)
    a_norm = np.sqrt(np.sum(a * a))
    b_norm = np.sqrt(np.sum(b * b, axis=-1))

    denominator = a_norm * b_norm
    return numerator/denominator

In [ ]:
def return_top_n_idx(question, n):
    question_vector = get_cls_token(question)
    sentence_similarity = {}
    for i in chatbot_q_vectors.keys():
        ir_vector = chatbot_q_vectors[i]
        similarity = custom_cosine_similarity(question_vector, ir_vector)
        sentence_similarity[i] = similarity

    sorted_sim = sorted(sentence_similarity.items(), key=lambda x: x[1], reverse=True)
    return sorted_sim[0:n]

In [ ]:
print(return_top_n_idx("오늘 너무 힘들어",5))

[(3285, array([[0.9760037]], dtype=float32)), (7121, array([[0.9664848]], dtype=float32)), (5947, array([[0.9598295]], dtype=float32)), (5959, array([[0.9573787]], dtype=float32)), (7176, array([[0.9529196]], dtype=float32))]


In [ ]:
print('most similar questions')
for result in return_top_n_idx('오늘 너무 힘들어',5):
  print(chatbot_Q[result[0]])

print()

print('most similar answer')
for result in return_top_n_idx('오늘 너무 힘들어',5):
  print(chatbot_A[result[0]])

most similar questions
오늘 너무 힘들다
오늘 너무 힘드네
너무 힘들어
너무나도 힘들어
오늘따라 너무 힘드네

most similar answer
고생 많았어요.
오늘은 힘내려 하지 말아요. 저에게 기대세요.
지금 무슨 말을 해도 와닿지 않겠지만 잘할 수 있을 거예요.
억지로라도 긍정적인 생각을 해보세요.
힘든 날이네요.


In [ ]:
# 정답이 아닌 경우

print('most similar questions')
for result in return_top_n_idx('너 이름이 뭐야?',5):
  print(chatbot_Q[result[0]])

print()

print('most similar answer')
for result in return_top_n_idx('너 이름이 뭐야?',5):
  print(chatbot_A[result[0]])

most similar questions
우정이 뭐야?
너 뭐니?
할 줄 아는거 뭐야?
사랑의 끝이 뭐야?
너 누구?

most similar answer
힘들 때 같이 있는 거요.
저는 위로봇입니다.
당신의 삶을 응원해 드릴 수 있어요라고 감히 말해 봅니다.
사랑하지 않는 것이죠.
저는 마음을 이어주는 위로봇입니다.


# 이진 분류 모델

In [ ]:
from transformers import BertForSequenceClassification

In [ ]:
MODEL_NAME = "/content/drive/MyDrive/1자연어처리/bert_two_sentence_classifier"
classifier_model = BertForSequenceClassification.from_pretrained(MODEL_NAME)
classifier_model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1

In [ ]:
def sentences_predict(sent_a, sent_b):
    classifier_model.eval()
    tokenized_sent = tokenizer(
            sent_a,
            sent_a,
            return_tensors="pt",
            truncation=True,
            add_special_tokens=True,
            max_length=64
    )

    tokenized_sent.to('cuda:0')

    with torch.no_grad():# 그라디엔트 계산 비활성화
        outputs = classifier_model(
            input_ids=tokenized_sent['input_ids'],
            attention_mask=tokenized_sent['attention_mask'],
            token_type_ids=tokenized_sent['token_type_ids']
            )

    logits = outputs[0]
    logits = logits.detach().cpu().numpy()
    result = np.argmax(logits)

    # if result == 0:
    #   result = 'non_similar'
    # elif result == 1:
    #   result = 'similar'
    return result

In [ ]:
print(sentences_predict('오늘 날씨가 어때요 ?', '오늘 날씨 알려줘'))
print(sentences_predict('오늘 날씨가 어때요 ?', '지금 날씨가 어때요?'))

1
1


# Top-N개의 문장을 입력으로 받으면 가장 적절한 문장을 반환하는 함수

In [ ]:
def get_answer(question, n):
    results = return_top_n_idx(question, n) # top n개를 list로
    for result in results: # n개를 반복문
        ir_answer = chatbot_A[result[0]]
        ir_question = chatbot_Q[result[0]]
        if sentences_predict(question, ir_question) == 1:  # 이진분류 모델이 query와 질문의 의미가 동일한지 확인
            return ir_answer # 정답 반환
    return chatbot_A[results[0][0]]

In [ ]:
print(get_answer("넌 이름이 뭐야?",5))

받는 것보다 주는 게 더 행복하고, 더 주고 싶은 마음인 것 같아요.


In [ ]:
print(get_answer("오늘 기분이 어때?",5))

동심으로 돌아갈 거 같아요.


In [ ]:
print(get_answer("어떻게 확인 하는데?",5))

객관적으로 생각해보세요.
